In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.tsatools import detrend

In [3]:
excel = pd.ExcelFile("Inflation-data.xlsx")

def find_cols_to_drop(df_t):
    df_t = df_t[197911 < df_t.index]
    df_t = df_t[df_t.index < 202000]

    mask = df_t.isna().rolling(window=3).sum() == 3
    return df_t,mask

In [11]:
hcpi_df = pd.read_excel(excel, "hcpi_m")
only_country_hcpi = hcpi_df[:-1].drop(columns=['IMF Country Code', 'Country', 'Indicator Type', 'Series Name', "Data source", "Note"])
df_t = only_country_hcpi.set_index("Country Code").T

_, mask = find_cols_to_drop(df_t)
cols_to_drop = mask.any()
df_cleaned = df_t.loc[:, ~cols_to_drop]

df_cleaned = df_cleaned[197911 < df_cleaned.index]
df_cleaned = df_cleaned[df_cleaned.index < 202000]

for col in df_cleaned:
    df_cleaned[col] = df_cleaned[col].astype("float64")

In [12]:
df_cleaned["AUT"]

197912     40.33883
198001     40.81597
198002     41.17383
198003     41.31300
198004     41.49193
            ...    
201908    106.49110
201909    106.99110
201910    107.19110
201911    107.39110
201912    108.09100
Name: AUT, Length: 481, dtype: float64

In [141]:
import numpy as np
import pandas as pd
from scipy.linalg import svd

def estimate_factors(X_df, r, max_iter=100, tol=1e-6):
    """
    Estimate missing values in X using the EM algorithm with factor analysis.
    
    Parameters:
    X_df : DataFrame (N, T)
        Panel data with missing values as np.nan.
    r : int
        Number of factors.
    max_iter : int, optional
        Maximum number of iterations.
    tol : float, optional
        Convergence tolerance.
    
    Returns:
    X_filled_df : DataFrame (N, T)
        Completed data matrix.
    F : DataFrame (T, r)
        Estimated factor matrix.
    """
    X = X_df.to_numpy()
    N, T = X.shape
    X_filled = X.copy()
    missing_mask = pd.isna(X)
    
    # Initialize missing values with column means
    col_means = np.nanmean(X, axis=0)
    X_filled[missing_mask] = np.take(col_means, np.where(missing_mask)[1])
    
    for _ in range(max_iter):
        X_old = X_filled.copy()
        
        # Compute SVD to get factor estimates
        # Compute covariance matrix
        S = (1 / N) * (X_filled.T @ X_filled)
        
        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = eigh(S)
        
        # Select r largest eigenvectors
        F = eigenvectors[:, -r:]  # (T, r)
        
        # Estimate factor loadings
        Lambda = np.linalg.lstsq(F, X_filled.T, rcond=None)[0].T  # (N, r)
        
        # Reconstruct the data matrix
        X_filled = Lambda @ F.T
        
        # Restore observed values
        X_filled[~missing_mask] = X[~missing_mask]
        
        # Check for convergence
        if np.linalg.norm(X_filled - X_old, ord='fro') / np.linalg.norm(X_old, ord='fro') < tol:
            break
    
    X_filled_df = pd.DataFrame(X_filled, index=X_df.index, columns=X_df.columns)
    F_df = pd.DataFrame(F, index=X_df.columns, columns=[f'Factor_{i+1}' for i in range(r)])
    
    return X_filled_df, F_df

X_imputed, F_df = estimate_factors(df_cleaned, r = 8)

In [143]:
X_imputed["SUR"].loc[201801:]

201801    275.125224
201802    277.849236
201803    278.268315
201804    278.896933
201805    279.123170
201806    285.351444
201807    282.459103
201808    284.554497
201809    285.811733
201810    287.068970
201811    287.697588
201812    288.326206
201901    290.002521
201902    289.792982
201903    289.792982
201904    291.678836
201905    293.145612
201906    293.983770
201907    294.821927
201908    295.869624
201909    297.336400
201910    299.222255
201911    300.060412
201912    300.479491
Name: SUR, dtype: float64

In [114]:
diff_df = df_cleaned.diff()
df_cleaned.iloc[0] + diff_df.cumsum()

Country Code,AUT,BDI,BEL,BFA,BHS,BOL,BRA,BRB,BWA,CAN,...,SWZ,SYC,THA,TTO,TUR,TWN,URY,USA,WSM,ZAF
197912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198001,40.81597,4.999283,38.20992,34.164901,34.609787,0.000195,1.884470e-10,27.985561,5.535150,33.02607,...,4.946693,25.886983,29.728047,10.076985,0.000895,43.15,0.015583,32.82465,12.033603,4.645056
198002,41.17383,4.949290,38.49874,34.016277,34.896292,0.000196,1.971480e-10,27.928738,5.734996,33.42112,...,4.940990,26.326530,30.597413,10.024440,0.001013,44.01,0.016269,33.28875,12.521641,4.691047
198003,41.31300,4.959289,38.56919,34.861575,35.220998,0.000199,2.090520e-10,28.411737,5.765276,33.65815,...,4.978055,26.465334,31.085105,10.036117,0.001052,44.25,0.016589,33.79504,12.758687,4.737038
198004,41.49193,5.019280,38.61850,34.675795,35.583904,0.000201,2.201030e-10,28.667443,5.771332,33.89518,...,4.949544,26.349664,31.594002,10.141207,0.001092,44.47,0.017015,34.17476,13.427995,4.783029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201908,106.49110,181.345782,107.96560,107.779563,116.556468,149.055890,1.213407e+02,135.991605,150.209102,108.08530,...,167.299329,130.174433,113.433925,143.413459,162.265700,98.58,205.213668,108.24460,119.497377,122.876100
201909,106.99110,182.988407,107.47010,107.556965,116.911037,148.934507,1.212922e+02,137.320742,150.356800,107.61130,...,167.478180,129.996806,113.544269,143.413459,163.877400,98.73,206.272159,108.32940,119.595890,123.281200
201910,107.19110,183.973981,107.85660,108.139959,116.384556,149.604490,1.214136e+02,137.880378,150.504498,107.92730,...,167.388191,131.728109,113.367718,143.413459,167.158500,98.75,207.823927,108.57700,119.373736,123.281200
201911,107.39110,185.288081,107.92600,107.334368,116.395300,151.272365,1.220328e+02,138.579923,150.652196,107.76930,...,167.959324,131.336731,113.224271,143.281524,167.795500,98.35,208.697439,108.51880,119.230653,123.416200
